## Домашняя работа
Группа: РИМ-150950

ФИО: Эрмиш Александр Александрович

## Задание 1
Реализуйте метакласс ThreadSafeSingleton, который обеспечивает создание только одного экземпляра класса, даже в многопоточной среде.
Используйте `from threading import Lock`


In [1]:
# Реализация метакласса ThreadSafeSingleton и класса DatabasePool для проверки

from threading import Lock  # Импортируем мьютекс для потокобезопасности
from typing import Any, Type
import random  # Импортируем random для генерации соединений

# Определяем метакласс ThreadSafeSingleton
class ThreadSafeSingleton(type):
    """
    Метакласс, реализующий потокобезопасный синглтон.
    Как работает:
    1. Каждый класс, использующий этот метакласс, получает собственный
       атрибут `_instance` (хранит единственный объект) и `_lock` (мьютекс).
    2. При вызове `MyClass()` выполнение попадает в `__call__`.
    3. Сначала проверяется, существует ли уже экземпляр.
       Если да – сразу возвращаем.
    4. Если нет – захватываем `_lock` и повторно проверяем.
    5. Если после захвата всё ещё нет экземпляра, создаём его обычным способом
       и сохраняем в `_instance`.
    6. Возвращаем полученный объект.
    """
    def __init__(cls, name: str, bases: tuple[type, ...], attrs: dict[str, Any]) -> None:  # Инициализация класса с этим метаклассом
        """
        При создании самого класса (не экземпляра) добавляем два «приватных» атрибута:
        - `_instance` – будет хранить единственный объект (по‑умолчанию None).
        - `_lock`    – глобальный мьютекс, общий для всех потоков,
                    пытающихся создать объект этого класса.
        """
        super().__init__(name, bases, attrs)  # Вызываем инициализацию базового type
        cls._instance = None  # Атрибут для хранения единственного экземпляра
        cls._lock = Lock()  # Мьютекс для синхронизации создания экземпляра
    
    def __call__(cls: Type, *args: Any, **kwargs: Any) -> Any:
        """
        Этот метод вызывается каждый раз, когда пользователь пишет `MyClass()`.
        Мы гарантируем, что будет создан НЕ БОЛЕЕ ОДНОГО объекта.
        """
        # Быстрая проверка без блокировки (экономит ресурсы после инициализации)
        if cls._instance is not None:
            return cls._instance
        # Захватываем блокировку, чтобы только один поток вошёл дальше.
        with cls._lock:
            # Внутри блока снова проверяем – возможно, другой поток уже успел создать объект.
            if cls._instance is None:
                # Стандартный вызов создаёт новый объект.
                cls._instance = super().__call__(*args, **kwargs)
        return cls._instance

# Класс с метаклассом-одиночкой
class DatabasePool(metaclass=ThreadSafeSingleton):
    """
    Пул соединений, реализованный как синглтон.
    Мы будем хранить список «свободных» соединений и выдавать
    следующее при запросе `get_connection`.
    """  
    def __init__(self): # Конструктор класса
        self._connections = set()  # Множество для хранения соединений
    def get_connection(self):  # Метод для получения нового соединения
        conn = random.randint(1000, 9999)  # Генерируем случайный номер соединения
        self._connections.add(conn)  # Добавляем соединение в множество
        return conn  # Возвращаем соединение

### проверка задания 1

In [4]:
# Создадим 3 экземпляра DatabasePool
pool1 = DatabasePool()  # Первый экземпляр
pool2 = DatabasePool()  # Второй экземпляр
pool3 = DatabasePool()  # Третий экземпляр

# Убедимся, что это один и тот же объект
assert pool1 is pool2 is pool3  # Проверка singleton: все переменные ссылаются на один объект
print("Все экземпляры pool1, pool2, pool3 — это один и тот же объект.")

# Проверим, что соединения разделяются между экземплярами
conn1 = pool1.get_connection()  # Получаем соединение через pool1
conn2 = pool2.get_connection()  # Получаем соединение через pool2
print(f"conn1: {conn1}, conn2: {conn2}")
assert conn1 != conn2  # Соединения должны быть разными

print("Соединения conn1 и conn2 различны")

Все экземпляры pool1, pool2, pool3 — это один и тот же объект.
conn1: 9279, conn2: 6060
Соединения conn1 и conn2 различны


## Задание 2

Создайте метакласс, который считает, сколько раз создавался каждый класс.

Требования:
1. Метакласс должен иметь атрибут _counters
2. При создании экземпляра класса счетчик должен увеличиваться
3. Добавьте метод get_count(), который возвращает количество созданных экземпляров

In [7]:
class CounterMeta(type):
    """Метакласс, автоматически считающий, сколько раз создан каждый класс."""
    def __new__(mcls, name, bases, namespace):
        """Создаём класс и сразу добавляем к нему метод get_count."""
        # Обычное создание класса через type.__new__
        cls = super().__new__(mcls, name, bases, namespace)
        # 1️⃣.1.  Гарантируем, что у метакласса есть общий словарь счётчиков.
        if not hasattr(mcls, "_counters"):
            mcls._counters: Dict[Type, int] = {}
        # 1️⃣.2.  Инициализируем счётчик для нового класса нулём.
        mcls._counters[cls] = 0
        # 1️⃣.3.  Добавляем classmethod get_count к самому классу.
        def get_count(inner_cls: Type) -> int:
            """Вернуть, сколько уже создано экземпляров данного класса."""
            return mcls._counters.get(inner_cls, 0)
        cls.get_count = classmethod(get_count)   # type: ignore[attr-defined]
        return cls

    def __call__(cls, *args, **kwargs):
        """Создаём объект и увеличиваем счётчик."""
        obj = super().__call__(*args, **kwargs)   # реальное создание
        # Увеличиваем счётчик именно для текущего класса
        type(cls)._counters[cls] += 1
        return obj

# Класс User с метаклассом CounterMeta
class User(metaclass=CounterMeta):
    def __init__(self, name):
        self.name = name

# Класс Product с метаклассом CounterMeta
class Product(metaclass=CounterMeta):
    def __init__(self, name):
        self.name = name

### проверка задания 2

In [8]:
# Проверка
user1 = User("Alice")
user2 = User("Bob")
product1 = Product("Laptop")

print(User.get_count())    # Должно быть 2
print(Product.get_count()) # Должно быть 1

2
1


In [ ]:
product1 = Product("IPhone")
print(User.get_count())    # Должно быть 2
print(Product.get_count()) # Должно быть 2

2
2


## Задание 3

Создайте метакласс, который автоматически добавляет метод describe() в каждый класс.

Требования:
1. Метод describe() должен возвращать строку с именем класса
1. Используйте метакласс для создания классов Car и Book

### проверка задания 3

In [15]:
# Метакласс, автоматически добавляющий метод describe()
class DescribeMeta(type):
    # __new__ вызывается *до* создания самого класса.
    def __new__(mcls, name, bases, attrs):
        def describe(self):
            # Метод describe возвращает строку вида
            # «Это объект класса <ИмяКласса>».
            return f"Это объект класса {self.__class__.__name__}"
        # Добавляем метод в набор атрибутов создаваемого класса
        attrs["describe"] = describe
        # Создаём класс обычным способом
        return super().__new__(mcls, name, bases, attrs)

# Класс Car с метаклассом DescribeMeta
class Car(metaclass=DescribeMeta):
    def __init__(self, brand):
        self.brand = brand

# Класс Book с метаклассом DescribeMeta
class Book(metaclass=DescribeMeta):
    def __init__(self, name):
        self.name = name

In [16]:
car = Car("Toyota")
book = Book("Python для начинающих")

print(car.describe())  # Должно быть "Это объект класса Car"
print(book.describe()) # Должно быть "Это объект класса Book"

Это объект класса Car
Это объект класса Book


## Задание 4

Создайте метакласс, который проверяет, что у класса есть метод save(). Можно использовать `__new__`

Требования:
1. Если у класса нет метода save(), метакласс должен выдать ошибку
1. Создайте класс User с методом save()
1. Попробуйте создать класс Message без метода save() (должна быть ошибка)

In [7]:
# Метакласс, проверяющий наличие метода save()
class SaveMeta(type):
    def __new__(mcs, name, bases, attrs):
        
        # Проверяем, объявлен ли метод `save` непосредственно в attrs.
        has_save = 'save' in attrs and callable(attrs['save'])
        
        # Если метода нет в текущем объявлении, ищем его у базовых классов.
        # Это позволяет наследовать `save` от уже корректных предков.
        if not has_save:
            for base in bases:
                if hasattr(base, 'save') and callable(getattr(base, 'save')):
                    has_save = True
                    break
        # Если после всех проверок метод все равно не найден - бросаем исключение
        if not has_save:
            raise TypeError(
                f"Класс '{name}' обязан определять метод 'save()'"
                f"(либо наследовать его от базового класса)."
            )
        
        # Если проверка прошла - создаем класс
        return super().__new__(mcs, name, bases, attrs)

# Класс User с методом save()
class User(metaclass=SaveMeta):
    def __init__(self, name):
        self.name = name
    def save(self):
        print(f'Пользователь {self.name} сохранён!')

### проверка задания 4

In [8]:
# Проверка
user = User("Alice")
user.save()  # Должно работать

# Этот код должен вызвать ошибку:
class Message(metaclass=SaveMeta):
    def __init__(self, text):
        self.text = text

Пользователь Alice сохранён!


TypeError: Класс 'Message' обязан определять метод 'save()'(либо наследовать его от базового класса).